In [1]:
import os
import pandas as pd
import requests
import re

from zipfile import ZipFile
from keplergl import KeplerGl
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Imports
- Residential Congrate Settings

In [2]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/matched_data/')
RCAC_oct0120_df = pd.read_csv('rcac_poi_match.csv')

In [3]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-03-2020/matched_data/')
RCAC_oct0220_df = pd.read_csv('rcac_poi_match.csv')

### Import Safegraph POI
- just the POI

In [6]:
os.chdir('/Volumes/USB30FD/RMDI_data/')

# filename = 'core_places_us_only/2020/08/Core-USA-August2020-Release-CORE_POI-2020_07-2020-08-07.zip'
filename = 'core_places_us_only/09142020/Core-USA-Sep-CORE_POI-2020_08-2020-09-08.zip'

poi = []

with ZipFile(filename, 'r') as zip:
    with zip.open('core_poi-part1.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part2.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part3.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part4.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))
    with zip.open('core_poi-part5.csv.gz') as myfile:
        poi.append(pd.read_csv(myfile, compression='gzip'))

poi_df = pd.concat(poi)

del poi

POI_DF = poi_df[poi_df['region']=='CA']

del poi_df

In [47]:
# POI_DF.shape # (704316, 18)
# POI_DF.head()

## Import Monthly POI data

In [8]:
files = ['monthly_places_patterns_May_2020_present/2020/10/07/02/patterns-part1.csv.gz',
        'monthly_places_patterns_May_2020_present/2020/10/07/02/patterns-part2.csv.gz',
        'monthly_places_patterns_May_2020_present/2020/10/07/02/patterns-part3.csv.gz',
        'monthly_places_patterns_May_2020_present/2020/10/07/02/patterns-part4.csv.gz']

september_month_df = []

for f in files:
    september_month_df.append(pd.read_csv(f, compression='gzip'))

SEPTEMBER_MONTH_DF = pd.concat(september_month_df)

del september_month_df, files

ca_september_month = SEPTEMBER_MONTH_DF[SEPTEMBER_MONTH_DF['region']=='CA']
del SEPTEMBER_MONTH_DF

In [9]:
ca_september_month.head(3)

,placekey,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type
5,zzw-222@5z5-t7m-wrk,sg:00558d9ab88d4ad9998e3bb43c3f6f60,The Insurance Place,2111 Palomar Airport Rd Ste 322,Carlsbad,CA,92011,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,13,1,"[1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,1,1,1...",6.073022e+10,{},{},{},{},NaN,10.0,"{""<5"":0,""5-20"":11,""21-60"":2,""61-240"":0,"">240"":0}",{},"{""The Joint"":100,""Z Ultimate Self Defense Stud...","[0,0,0,0,0,0,0,0,0,0,9,9,0,0,0,0,0,0,0,0,0,0,0,0]","{""Monday"":2,""Tuesday"":2,""Wednesday"":1,""Thursda...","{""android"":4,""ios"":0}"
21,zzw-227@5z4-ztv-2rk,sg:01ba472a875d402e95738e6a93cd6e2f,Uplifters Kitchen,2819 Ocean Park Blvd,Santa Monica,CA,90405,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,67,27,"[3,3,4,3,2,5,1,4,3,3,2,2,3,3,2,1,2,1,4,1,0,2,2...",6.037702e+10,"{""060014506025"":4,""060372722012"":4,""0603723470...","{""060372722012"":4,""490111269024"":4,""0603713472...",{},"{""US"":25}",16014.0,127.0,"{""<5"":1,""5-20"":19,""21-60"":6,""61-240"":33,"">240"":8}",{},"{""Starbucks"":26,""Ralphs"":22,""76"":18,""The Home ...","[0,1,1,0,0,0,0,1,2,26,27,29,39,25,24,22,5,2,3,...","{""Monday"":4,""Tuesday"":14,""Wednesday"":12,""Thurs...","{""android"":10,""ios"":18}"
22,224-222@5z6-3nv-9vf,sg:01bd97e5e4d34ca5a3fa3521e515495a,Holy Cross Lutheran Church,4321 Cerritos Ave,Cypress,CA,90630,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,18,16,"[0,1,0,0,0,0,1,0,0,0,0,0,4,5,0,0,0,0,0,1,3,0,0...",6.059110e+10,"{""060591101042"":4,""060591100081"":4,""0603753610...","{""060591101063"":6,""060591101042"":5,""0605911000...",{},"{""US"":17}",3202.0,50.0,"{""<5"":0,""5-20"":4,""21-60"":6,""61-240"":6,"">240"":2}","{""Subway"":24}","{""Target"":56,""Costco Wholesale Corp."":40,""Subw...","[0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,2,7,13,8,4,1,...","{""Monday"":11,""Tuesday"":0,""Wednesday"":1,""Thursd...","{""android"":11,""ios"":4}"


### Import community data from Los Angeles Almanac

In [6]:
url = 'http://www.laalmanac.com/communications/cm02_communities.php'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
td_data = soup.find_all('td')


# Extract comminity and zip code data
communities = {}
zipcodes = {}
c = 1
for index in range(len(td_data)):
    if (c % 2) != 0:
        community = td_data[index].text.strip()
        idx = int(index/2)
        communities[idx] = community
    if (c % 2) != 0:
        zip_code = td_data[index+1].text.strip()
        idx = int(index/2)
        zipcodes[idx] = zip_code
    c+=1

In [7]:
la_communities_n_zip = []
cities = []
cities_zips = []
# cities_zips_df = []


for i in zipcodes:
    la_communities_n_zip.append(communities[i] + '---' + zipcodes[i])


for i in la_communities_n_zip:
    city = i.split('---')[0]
    # Below currently in use
    item = re.sub(r"(^Los.Angeles.|\(Los Angeles\)|PO Boxes|\/.*)", "", city.strip())
    
    item = re.sub(r"(^Pasadena.*)", "Pasadena", item)
    item = re.sub(r"(^Alhambra.*)", "Alhambra", item)
    item = re.sub(r"(^Downtown.*)", "Downtown", item)
    item = re.sub(r"(.*Long Beach.*)", "Long Beach", item)
    item = re.sub(r"(Santa Clarita )", "", item)

    # these are mine
    item = re.sub(r"(Rancho Dominguez.*)", "West Rancho Dominguez", item) # Officially 'West Rancho Dominguez'
    item = re.sub(r"(Los Angeles International Airport.*)", "Los Angeles", item) # ME: get's 'Los Angeles' 
    
    item = re.sub(r"(\(|\))", "", item.strip())
    
    # this is mine
    item = re.sub(r" $","", item)
    
    cities.append(item)
    
    zipcode = i.split('---')[1]
    cities_zips.append(zipcode)


In [8]:
values = list()
value_set = set()
for key, val in communities.items():
    # Below currently in use
    item = re.sub(r"(^Los.Angeles.|\(Los Angeles\)|PO Boxes|\/.*)", "", val.strip())
    
    item = re.sub(r"(^Pasadena.*)", "Pasadena", item)
    item = re.sub(r"(^Alhambra.*)", "Alhambra", item)
    item = re.sub(r"(^Downtown.*)", "Downtown", item)
    item = re.sub(r"(.*Long Beach.*)", "Long Beach", item)
    item = re.sub(r"(Santa Clarita )", "", item)

    # these are mine
    item = re.sub(r"(Rancho Dominguez.*)", "West Rancho Dominguez", item) # Officially 'West Rancho Dominguez'
    item = re.sub(r"(Los Angeles International Airport.*)", "Los Angeles", item) # ME: get's 'Los Angeles' 
    
    item = re.sub(r"(\(|\))", "", item.strip())
    
    # this is mine
    item = re.sub(r" $","", item)
    
    values.append(item)

values = set(values)
values = list(values)
LA_communities_df = pd.DataFrame(values)
LA_communities_df.columns = ['city']
del values

In [9]:
len(LA_communities_df)

370

## Exploring imported data

In [10]:
RCAC_oct0120_df.shape

(467, 13)

In [11]:
RCAC_oct0120_df.sort_values('number_of_candidate_matches', ascending=False)

,customer_location_name,customer_city_name,customer_number_of_confirmed_staff,customer_number_of_confirmed_residents,customer_total_deaths,customer_city,customer_state,customer_postal_code,other_match_candidates,number_of_candidate_matches,warnings,is_closed,safegraph_place_id
377,The Village at Sherman Oaks,"Sherman Oaks, CA",1,3,0,Sherman Oaks,CA,91403,"sg:55bc747692a8405194883766beac2c6b,sg:ce3c236...",8,MISSING_STREET_ADDRESS,False,sg:1dfed36a59ff4b59b23c3a62eae9198e
376,The Village at Sherman Oaks,"Sherman Oaks, CA",1,3,0,Sherman Oaks,CA,91403,"sg:55bc747692a8405194883766beac2c6b,sg:ce3c236...",8,MISSING_STREET_ADDRESS,False,sg:1dfed36a59ff4b59b23c3a62eae9198e
64,Brookdale Assisted Living North Tarzana RCFE,"Tarzana, CA",6,4,1,Tarzana,CA,91356,"sg:a787de65f2694e06baa9c402396aee1a,sg:0e11001...",4,MISSING_STREET_ADDRESS,False,sg:4baf4c41136045b2945572b6196425b4
66,Brookdale Assisted Living North Tarzana RCFE,"Tarzana, CA",6,4,1,Tarzana,CA,91356,"sg:a787de65f2694e06baa9c402396aee1a,sg:0e11001...",4,MISSING_STREET_ADDRESS,False,sg:4baf4c41136045b2945572b6196425b4
228,Kei-Ai Los Angeles Healthcare Center,"Los Angeles, CA",37,25,8,Los Angeles,CA,90045,"sg:75838b09fe99470080be9a534f004098,sg:8f3f540...",3,MISSING_STREET_ADDRESS,False,sg:f5ad31b930034c858e251f10fbfbbcb5
346,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",28,54,13,Sherman Oaks,CA,91403,"sg:736d114309144697af0887faee8648c1,sg:e02440d...",3,MISSING_STREET_ADDRESS,False,sg:55bc747692a8405194883766beac2c6b
345,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",28,54,13,Sherman Oaks,CA,91403,"sg:736d114309144697af0887faee8648c1,sg:e02440d...",3,MISSING_STREET_ADDRESS,False,sg:55bc747692a8405194883766beac2c6b
226,Jackman Street House (Tarzana Treatment Center),"Lancaster, CA",0,3,0,Lancaster,CA,93534,"sg:4e56f5f6fed5458db22e93b33256e493,sg:f14b9c3...",3,MISSING_STREET_ADDRESS,False,sg:d67d278f939b4c2a8f199035abe60bc3
172,Glendora Grand Convalescent Hospital,"Glendora, CA",91,240,16,Glendora,CA,91740,sg:f1295a9bd60947f8b2c52f85d2320171,2,MISSING_STREET_ADDRESS,False,sg:70da55eeae3244e3a91b5f047ff650e1
251,Motion Picture & Television Fund,"Woodland Hills, CA",16,17,7,Woodland Hills,CA,91302,sg:4c7434e9d398436a9581eea35372fac3,2,MISSING_STREET_ADDRESS,False,sg:c13b2437a37941c4a6cc7cd4939011df


In [12]:
## 467 rows
len(RCAC_oct0120_df['customer_location_name'])

467

In [13]:
## 349 have 1 match
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['customer_location_name']) 


349

In [14]:
## 18 have > 1 match
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']>1]['customer_location_name']) 

18

In [15]:
## 100 POI are not found in safegraph
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==0]['customer_location_name']) 


100

### Get unique count of POI names

In [16]:
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['customer_location_name'].unique())


156

### Get unique count of POI id

In [17]:
len(RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]['safegraph_place_id'].unique())


154

In [77]:
# RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1].groupby('safegraph_place_id').count()


## Investigate merge data between imported matched cases and SAFEGRAPH POI

In [19]:
la_poi = pd.merge(POI_DF, LA_communities_df, on='city', how='inner')
la_poi.to_csv('/Volumes/USB30FD/RMDI_data/la_poi.csv')
la_poi = pd.read_csv('/Volumes/USB30FD/RMDI_data/la_poi.csv')

In [41]:
la_september_month = pd.merge(ca_september_month, LA_communities_df, on='city', how='inner')
la_september_month.to_csv('/Volumes/USB30FD/RMDI_data/la_september_month.csv')
la_september_month = pd.read_csv('/Volumes/USB30FD/RMDI_data/la_september_month.csv')

In [78]:
single_matches = RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]
# single_matches.to_csv('/Volumes/USB30FD/RMDI_data/single_matches.csv')
# single_matches = pd.read_csv('/Volumes/USB30FD/RMDI_data/single_matches.csv')

In [79]:
final_matches = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')[['customer_location_name','customer_city_name','customer_number_of_confirmed_staff','customer_number_of_confirmed_residents','customer_total_deaths','customer_city','customer_state','customer_postal_code','safegraph_place_id','latitude','longitude','street_address','open_hours']]
final_matches.shape # 349



(349, 13)

In [80]:
df1_map = KeplerGl(height=600, width=800)
df1_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [81]:
df1_map.add_data(data=final_matches, name='POI Cases in Los Angeles')

In [82]:
df1_map.save_to_html(file_name='Residential_Congregate_Settings.html')

Map saved to Residential_Congregate_Settings.html!


## Data Imports
- Non-Residential Settings

In [56]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/matched_data/')
NON_RES_oct0120_df = pd.read_csv('non_res_poi_match.csv')

In [75]:
# NON_RES_oct0120_df.head()
single_matches = NON_RES_oct0120_df[NON_RES_oct0120_df['number_of_candidate_matches']==1]

In [84]:
final_matches = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')
# final_matches.head()

In [77]:
df1_map = KeplerGl(height=600, width=800)
df1_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [71]:
df1_map.add_data(data=final_matches, name='POI Cases in Los Angeles')

In [74]:
df1_map.save_to_html(file_name='Non-Residential_Settings.html')

Map saved to Non-Residential Settings.html!


## Data Imports
- Education 

In [85]:
!pwd

/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/matched_data


In [86]:
os.chdir('/Volumes/USB30FD/LA-HEALTH-COVID-Data/data_10-02-2020/matched_data/')
EDUCATION_oct0120_df = pd.read_csv('education_poi_match.csv')

In [88]:
single_matches = EDUCATION_oct0120_df[EDUCATION_oct0120_df['number_of_candidate_matches']==1]

In [90]:
final_matches = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')
# final_matches.head()

In [93]:
df1_map = KeplerGl(height=600, width=800)
df1_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [94]:
df1_map.add_data(data=final_matches, name='POI Cases in Los Angeles')

In [95]:
df1_map.save_to_html(file_name='Education_settings.html')

Map saved to Education_settings.html!


## Union all three datasets

In [96]:
single_matches = RCAC_oct0120_df[RCAC_oct0120_df['number_of_candidate_matches']==1]
final_single_matches_rcac = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')[['customer_location_name','customer_city_name','customer_number_of_confirmed_staff','customer_number_of_confirmed_residents','customer_total_deaths','customer_city','customer_state','customer_postal_code','safegraph_place_id','latitude','longitude','street_address','open_hours']]


In [ ]:
# del final_single_matches_rcac['customer_postal_code']
# final_single_matches_rcac.head()
final_single_matches_rcac = pd.DataFrame.drop_duplicates(final_single_matches_rcac)
final_single_matches_rcac['total_cases'] = final_single_matches_rcac['customer_number_of_confirmed_staff'] + final_single_matches_rcac['customer_number_of_confirmed_residents']
rcac_source = final_single_matches_rcac[['customer_location_name', 'latitude', 'longitude','total_cases']]


In [97]:
single_matches = NON_RES_oct0120_df[NON_RES_oct0120_df['number_of_candidate_matches']==1]
final_single_matches_nonres = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')


In [176]:
# del final_single_matches_nonres['customer_zipcode']
# final_single_matches_nonres.head() # (82, 29)
# final_single_matches_nonres.groupby(['customer_location_name', 'safegraph_place_id'])
# final_single_matches_rcac = pd.DataFrame.drop_duplicates(final_single_matches_nonres)
nonres_source = final_single_matches_nonres[['customer_location_name', 'latitude', 'longitude', 'customer_total_confirmed_staff']]


In [98]:
single_matches = EDUCATION_oct0120_df[EDUCATION_oct0120_df['number_of_candidate_matches']==1]
final_single_matches_education = pd.merge(single_matches, la_poi, on=['safegraph_place_id'], how='left')


In [185]:
# final_single_matches_education # (5, 30)
# final_single_matches_education['total_cases'] = final_single_matches_education['customer_total_confirmed_staff'] + final_single_matches_education['customer_total_confirmed_students']
education_source = final_single_matches_education[['customer_location_name','latitude', 'longitude', 'total_cases']]


In [197]:
# rcac_source.columns = ['POI', 'Latitude', 'Longitude', 'total_cases']
rcac_source.head() # (162, 4) 
# nonres_source.columns = ['POI', 'Latitude', 'Longitude', 'total_cases']
nonres_source.head() # (82, 4)
# education_source.columns = ['POI', 'Latitude', 'Longitude', 'total_cases']
education_source.head() # (5, 4)

,POI,Latitude,Longitude,total_cases
0,La Petite Academy Lancaster,34.677914,-118.113120,10
1,Life Pacific University Event,34.098316,-117.830050,14
2,Mulberry Tree Learning Center,33.960991,-118.025852,5
3,Temple Beth Hillel Nursery School,34.157102,-118.401679,4
4,Toyota Sports Performance Center Camp,33.921763,-118.388372,5


In [209]:
test = pd.concat([rcac_source, nonres_source, education_source], axis=0)

In [214]:
test.head()

,POI,Latitude,Longitude,total_cases
0,Affinity Healthcare Center,33.889519,-118.175942,12
1,Alameda Care Center,34.163811,-118.313025,6
3,Alcott Rehabilitation Hospital,34.052867,-118.317157,32
4,"Alhambra Healthcare & Wellness Centre, LP",34.090421,-118.124228,4
7,Angels Nursing Health Center,34.059515,-118.268887,5


In [ ]:
im here - now need to find day over day deltas

## Explore mergings

In [35]:
oct1_oct2_df = pd.merge(RCAC_oct0120_df, RCAC_oct0220_df, on=['location_name', 'city_name'], how='inner')


In [57]:
RCAC_oct0220_df[RCAC_oct0220_df['location_name']=='Elwyn El Monte']

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
62,Elwyn El Monte,"El Monte, CA",8,1,0


In [58]:
RCAC_oct0120_df[RCAC_oct0120_df['location_name']=='Elwyn El Monte']

,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
65,Elwyn El Monte,"El Monte, CA",6,1,0


In [111]:
oct1_oct2_df.head(2)

,location_name,city_name,number_of_confirmed_staff_x,number_of_confirmed_residents_x,total_deaths_x,number_of_confirmed_staff_y,number_of_confirmed_residents_y,total_deaths_y
0,A and M Armour Home Care,"Pomona, CA",7,4,0,7,4,0
1,Affinity Healthcare Center,"Paramount, CA",6,6,0,6,6,0


In [82]:
today = date.today()
today = today - timedelta(days=1)
today = today.strftime("%m-%d-%Y")

In [83]:
print(today)

10-02-2020


In [109]:
oct2_join_df = pd.DataFrame(columns=['date', 'location_name', 'city', 'confirmed_staff_delta', 'confirmed_residents_delta', 'total_deaths_delta'])

oct2_join_df['location_name'] = oct1_oct2_df['location_name']
oct2_join_df['city'] = oct1_oct2_df['city_name']
oct2_join_df['confirmed_staff_delta'] = oct1_oct2_df['number_of_confirmed_staff_y'] - oct1_oct2_df['number_of_confirmed_staff_x']
oct2_join_df['confirmed_residents_delta'] = oct1_oct2_df['number_of_confirmed_residents_y'] - oct1_oct2_df['number_of_confirmed_residents_x']
oct2_join_df['total_deaths_delta'] = oct1_oct2_df['total_deaths_y'] - oct1_oct2_df['total_deaths_x']
oct2_join_df['date'] = today
oct2_join_df.sort_values('confirmed_staff_delta', ascending=False)


,date,location_name,city,confirmed_staff_delta,confirmed_residents_delta,total_deaths_delta
186,10-02-2020,Windsor Gardens Convalescent Hospital,"Los Angeles, CA",2,1,2
61,10-02-2020,Elwyn El Monte,"El Monte, CA",2,0,0
143,10-02-2020,Sherman Oaks Health & Rehabilitation Center,"Sherman Oaks, CA",1,0,0
70,10-02-2020,Golden Legacy (Previously San Fernando Post Ac...,"Sylmar, CA",1,0,0
31,10-02-2020,Buena Ventura Post Acute Care Center,"Los Angeles, CA",1,0,0
20,10-02-2020,Beacon Healthcare Center,"West Covina, CA",1,1,0
159,10-02-2020,Topanga Terrace Convalescent Center,"West Hills, CA",1,1,0
43,10-02-2020,Claremont Care Center,"Pomona, CA",1,0,0
131,10-02-2020,Rinaldi Convalescent Hospital,"Granada Hills, CA",0,0,0
125,10-02-2020,Pathways Sober Living,"Pomona, CA",0,0,0


In [129]:
oct1_oct2_df = pd.merge(RCAC_oct0120_df, RCAC_oct0220_df, on=['location_name', 'city_name'], how='inner')

RCAC_oct0220_df[(~RCAC_oct0220_df['location_name'].isin(oct1_oct2_df['location_name']))&(~RCAC_oct0220_df['city_name'].isin(oct1_oct2_df['city_name']))]



,location_name,city_name,number_of_confirmed_staff,number_of_confirmed_residents,total_deaths
31,Brookfield Healthcare Center,"Downey, CA",6,18,6
109,Mother Gertrude Home Assisted Living,"San Fernando, CA",2,10,1


In [124]:
oct2_new_df = pd.DataFrame(columns=['date', 'location_name', 'city', 'confirmed_staff_delta', 'confirmed_residents_delta', 'total_deaths_delta'])

oct_2_new_df.shape
oct2_new_df['location_name'] = oct1_oct2_df['location_name']
oct2_new_df['city'] = oct1_oct2_df['city_name']
oct2_new_df['confirmed_staff_delta'] = oct1_oct2_df['number_of_confirmed_staff_y'] - oct1_oct2_df['number_of_confirmed_staff_x']
oct2_new_df['confirmed_residents_delta'] = oct1_oct2_df['number_of_confirmed_residents_y'] - oct1_oct2_df['number_of_confirmed_residents_x']
oct2_new_df['total_deaths_delta'] = oct1_oct2_df['total_deaths_y'] - oct1_oct2_df['total_deaths_x']
oct2_new_df['date'] = today
oct2_new_df.sort_values('confirmed_staff_delta', ascending=False)


In [18]:
sample = ca_september_month.head(50)
sample

,placekey,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type
5,zzw-222@5z5-t7m-wrk,sg:00558d9ab88d4ad9998e3bb43c3f6f60,The Insurance Place,2111 Palomar Airport Rd Ste 322,Carlsbad,CA,92011,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,13,1,"[1,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,1,1,1...",6.073022e+10,{},{},{},{},NaN,10.0,"{""<5"":0,""5-20"":11,""21-60"":2,""61-240"":0,"">240"":0}",{},"{""The Joint"":100,""Z Ultimate Self Defense Stud...","[0,0,0,0,0,0,0,0,0,0,9,9,0,0,0,0,0,0,0,0,0,0,0,0]","{""Monday"":2,""Tuesday"":2,""Wednesday"":1,""Thursda...","{""android"":4,""ios"":0}"
21,zzw-227@5z4-ztv-2rk,sg:01ba472a875d402e95738e6a93cd6e2f,Uplifters Kitchen,2819 Ocean Park Blvd,Santa Monica,CA,90405,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,67,27,"[3,3,4,3,2,5,1,4,3,3,2,2,3,3,2,1,2,1,4,1,0,2,2...",6.037702e+10,"{""060014506025"":4,""060372722012"":4,""0603723470...","{""060372722012"":4,""490111269024"":4,""0603713472...",{},"{""US"":25}",16014.0,127.0,"{""<5"":1,""5-20"":19,""21-60"":6,""61-240"":33,"">240"":8}",{},"{""Starbucks"":26,""Ralphs"":22,""76"":18,""The Home ...","[0,1,1,0,0,0,0,1,2,26,27,29,39,25,24,22,5,2,3,...","{""Monday"":4,""Tuesday"":14,""Wednesday"":12,""Thurs...","{""android"":10,""ios"":18}"
22,224-222@5z6-3nv-9vf,sg:01bd97e5e4d34ca5a3fa3521e515495a,Holy Cross Lutheran Church,4321 Cerritos Ave,Cypress,CA,90630,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,18,16,"[0,1,0,0,0,0,1,0,0,0,0,0,4,5,0,0,0,0,0,1,3,0,0...",6.059110e+10,"{""060591101042"":4,""060591100081"":4,""0603753610...","{""060591101063"":6,""060591101042"":5,""0605911000...",{},"{""US"":17}",3202.0,50.0,"{""<5"":0,""5-20"":4,""21-60"":6,""61-240"":6,"">240"":2}","{""Subway"":24}","{""Target"":56,""Costco Wholesale Corp."":40,""Subw...","[0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,2,7,13,8,4,1,...","{""Monday"":11,""Tuesday"":0,""Wednesday"":1,""Thursd...","{""android"":11,""ios"":4}"
32,228-222@5z6-3ky-4y9,sg:02b54fb3e3a14f7bab06f145c30bddb6,WorldWide Hospitality,7311 Madison St Ste D,Paramount,CA,90723,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,62,38,"[2,1,1,2,1,4,0,3,3,8,2,0,1,4,4,2,3,1,1,2,1,2,3...",6.037554e+10,"{""060375362004"":4,""060372083011"":4,""0605909970...","{""060375543022"":4,""060375538012"":4,""0603755451...",{},"{""US"":29}",5046.0,25.0,"{""<5"":2,""5-20"":26,""21-60"":15,""61-240"":10,"">240...","{""Round Table Pizza"":100,""Jetro Cash & Carry"":12}","{""76"":20,""ARCO"":20,""Jack in the Box"":17,""McDon...","[8,7,8,9,9,9,11,12,19,17,12,12,14,11,11,6,6,12...","{""Monday"":6,""Tuesday"":12,""Wednesday"":13,""Thurs...","{""android"":24,""ios"":12}"
34,zzw-222@5vg-56b-y7q,sg:02d70f5c672d42dbbcd350498edec836,Sahouria Pediatric Dentistry,1303 Medical Center Dr,Rohnert Park,CA,94928,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,47,33,"[3,3,4,0,0,1,0,2,6,0,1,0,0,3,2,2,6,0,0,0,0,2,1...",6.097151e+10,"{""060971513093"":5,""060971531032"":4,""0609715120...","{""060971513064"":4,""060971513114"":4,""0609715130...",{},"{""US"":32}",2316.0,30.0,"{""<5"":0,""5-20"":16,""21-60"":17,""61-240"":14,"">240...",{},"{""Target"":45,""Safeway"":33,""Starbucks"":29,""Cost...","[0,0,0,0,0,1,0,0,3,6,7,5,5,8,12,20,13,2,1,1,0,...","{""Monday"":7,""Tuesday"":11,""Wednesday"":14,""Thurs...","{""android"":4,""ios"":31}"
44,227-223@5z6-9bs-qpv,sg:03b0e2c511cb46418f8e95f942f2b12a,The Gorilla Cookie,68176 30th Ave,Cathedral City,CA,92234,NaN,NaN,2020-09-01T00:00:00-07:00,2020-10-01T00:00:00-07:00,3,3,"[0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0...",6.065045e+10,{},{},{},{},NaN,6.0,"{""<5"":0,""5-20"":2,""21-60"":0,""61-240"":0,"">2

In [19]:
sample.to_csv('CA_sample.csv', index = False)
# !pwd